In [ ]:
# Import Libraries
import pandas as pd
import numpy as np

In [ ]:
# Upload CSV File
from google.colab import files
uploaded = files.upload()

Saving railway - Copy (sql).csv to railway - Copy (sql).csv


In [ ]:
# Read CSV File
df = pd.read_csv("railway - Copy (sql).csv")

In [ ]:
# Check Nulls
df.isnull().sum()

,0
Transaction ID,0
Date of Purchase,0
Time of Purchase,0
Purchase Type,0
Payment Method,0
Railcard,20918
Ticket Class,0
Ticket Type,0
Price,0
Departure Station,0


In [ ]:
# Check Duplicates In Transaction ID
df[df.duplicated(subset=['Transaction ID'], keep=False)]

,Transaction ID,Date of Purchase,Time of Purchase,Purchase Type,Payment Method,Railcard,Ticket Class,Ticket Type,Price,Departure Station,Arrival Destination,Date of Journey,Departure Time,Arrival Time,Actual Arrival Time,Journey Status,Reason for Delay,Refund Request


In [ ]:
# Change Datatype To String
df['Transaction ID'] = df['Transaction ID'].astype(str)

In [ ]:
# Change Datatype To String
TEXT_COLS = ['Purchase Type', 'Payment Method', 'Railcard', 'Ticket Class',
             'Ticket Type', 'Departure Station', 'Arrival Destination',
             'Reason for Delay', 'Journey Status', 'Refund Request']
for col in TEXT_COLS:
    df[col] = df[col].astype(str).str.strip()

In [ ]:
# Change Datatype To Datetime
df['Date of Journey'] = pd.to_datetime(df['Date of Journey'], errors='coerce')
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')
df['Departure Time'] = pd.to_datetime(df['Departure Time'], format='%H:%M:%S', errors='coerce').dt.time
df['Arrival Time'] = pd.to_datetime(df['Arrival Time'], format='%H:%M:%S', errors='coerce').dt.time
df['Actual Arrival Time'] = pd.to_datetime(df['Actual Arrival Time'], format='%H:%M:%S', errors='coerce').dt.time

In [ ]:
# Create New Columns
df['Scheduled Datetime'] = df.apply(lambda row: pd.to_datetime(f"{row['Date of Journey'].date()} {row['Arrival Time']}", errors='coerce'), axis=1)
df['Actual Datetime'] = df.apply(lambda row: pd.to_datetime(f"{row['Date of Journey'].date()} {row['Actual Arrival Time']}", errors='coerce')
                                 if pd.notna(row['Actual Arrival Time']) else pd.NaT, axis=1)
df['Delay Time'] = df['Actual Datetime'] - df['Scheduled Datetime']
df['Delay in Minutes (Numeric)'] = df['Delay Time'].dt.total_seconds() / 60


In [ ]:
# Replace Missing Values
df['Reason for Delay'].replace('nan', 'Not Delayed', inplace=True)
df['Delay in Minutes (Numeric)'].replace('nan',0, inplace=True)
df['Railcard'].replace('nan','None',inplace=True)

/tmp/ipython-input-3185650407.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Reason for Delay'].replace('nan', 'Not Delayed', inplace=True)
/tmp/ipython-input-3185650407.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

In [ ]:
# Replace 'Weather Conditions' by 'Weather'
df['Reason for Delay'] = df['Reason for Delay'].replace('Weather Conditions', 'Weather')

In [ ]:
# Calculate Journey Duration (Scheduled)
df['Journey Duration (Scheduled)'] = df['Scheduled Datetime'] - pd.to_datetime(df['Date of Journey'].astype(str) + ' ' + df['Departure Time'].astype(str))
df['Journey Duration (Min)'] = df['Journey Duration (Scheduled)'].dt.total_seconds() / 60

In [ ]:
# Create Day of Week
df['Day of Week'] = df['Date of Journey'].dt.day_name()
# Create Routes
df['Routes'] = df['Departure Station'] + ' → ' + df['Arrival Destination']

In [ ]:
# Create Refund Amount
def calculate_refund(row):
    is_cancelled = pd.isna(row['Actual Arrival Time'])
    is_requested = row['Refund Request'].upper() == 'YES'
    delay = row['Delay in Minutes (Numeric)']

    if not is_requested:
        return 0.0


    if is_cancelled or delay > 0:
        return row['Price']
    else:
        return 0.0

df['Refund Amount'] = df.apply(calculate_refund, axis=1)

In [ ]:
# Generate a unique ID for each unique journey type
df['Unique Journey Key'] = df['Routes'] + df['Date of Journey'].astype(str)

In [ ]:
# Map Unique Journey Key to a simple sequential Journey ID
unique_journey_keys = df[['Unique Journey Key']].drop_duplicates().sort_values(by='Unique Journey Key').reset_index(drop=True)
unique_journey_keys.insert(0, 'Journey ID', unique_journey_keys.index + 1)
df = pd.merge(df, unique_journey_keys, on='Unique Journey Key', how='left')

In [ ]:
# Data Normalization #

# Create Dimension Table : Delay Reason

# 1. Extract unique delay reasons
df_delay_reasons = df[['Reason for Delay']].drop_duplicates().reset_index(drop=True)

# 2. Define category mapping function (fixed for lowercase matching)
def map_delay_category(reason):
    reason = str(reason).lower()  # ensure it's a string and lowercase
    if 'signal failure' in reason:
        return 'Infrastructure'
    elif 'technical issue' in reason:
        return 'Rolling Stock'
    elif 'weather' in reason or 'traffic' in reason:
        return 'External Factors'
    elif 'staffing' in reason or 'staff shortage' in reason:
        return 'Operational'
    elif reason == 'not delayed':
        return 'No Delay'
    else:
        return 'Other'

# 3. Apply category mapping
df_delay_reasons['Category'] = df_delay_reasons['Reason for Delay'].apply(map_delay_category)

# 4. Add Delay ID
df_delay_reasons.insert(0, 'Delay ID', df_delay_reasons.index + 1)

# 5. Rename column
df_delay_reasons.rename(columns={'Reason for Delay': 'Reason for Delay (Text)'}, inplace=True)

In [ ]:
# Create Dimension Table : Stations
dep_stations = df[['Departure Station']].rename(columns={'Departure Station': 'Station Name'})
arr_stations = df[['Arrival Destination']].rename(columns={'Arrival Destination': 'Station Name'})
df_stations = pd.concat([dep_stations, arr_stations]).drop_duplicates().sort_values(by='Station Name').reset_index(drop=True)
df_stations.insert(0, 'Station ID', df_stations.index + 1)

In [ ]:
# Create Dimension Table : Tickets (Ticket Details)
df_tickets = df[['Railcard', 'Ticket Class', 'Ticket Type']].drop_duplicates().reset_index(drop=True)
df_tickets.insert(0, 'Ticket ID', df_tickets.index + 1)
df_tickets['Ticket'] = df_tickets['Ticket Type'] + '|' + df_tickets['Ticket Class'] + '|' + df_tickets['Railcard']

In [ ]:
# PREP: Ensure all column names are clean
df.columns = df.columns.str.strip()
df_delay_reasons.columns = df_delay_reasons.columns.str.strip()
df_stations.columns = df_stations.columns.str.strip()
df_tickets.columns = df_tickets.columns.str.strip()

# Link Journeys to Delay Reasons

# Step 0: Remove any old 'Delay ID' columns to prevent duplicates
df = df.loc[:, ~df.columns.str.contains('Delay ID', case=False)]

# Step 1: Clean column names
df.columns = df.columns.str.strip()
df_delay_reasons.columns = df_delay_reasons.columns.str.strip()

# Step 2: Merge cleanly on reason text
df = pd.merge(
    df,
    df_delay_reasons[['Delay ID', 'Reason for Delay (Text)']],
    left_on='Reason for Delay',
    right_on='Reason for Delay (Text)',
    how='left'
)

# Step 3: Rename and drop unnecessary columns
df.rename(columns={'Delay ID': 'Delay ID FK'}, inplace=True)

# Drop only if the columns exist
for col in ['Reason for Delay', 'Reason for Delay (Text)', 'Category']:
    if col in df.columns:
        df.drop(columns=col, inplace=True)

# Link Journeys to Stations (Departure and Arrival)

# Departure Station
df = pd.merge(
    df,
    df_stations[['Station ID', 'Station Name']],
    left_on='Departure Station',
    right_on='Station Name',
    how='left'
)
df.rename(columns={'Station ID': 'Dep ID'}, inplace=True)
if 'Departure Station' in df.columns:
    df.drop(columns=['Departure Station'], inplace=True)
if 'Station Name' in df.columns:
    df.drop(columns=['Station Name'], inplace=True)

# Arrival Destination
df = pd.merge(
    df,
    df_stations[['Station ID', 'Station Name']],
    left_on='Arrival Destination',
    right_on='Station Name',
    how='left'
)
df.rename(columns={'Station ID': 'Arr ID'}, inplace=True)
if 'Arrival Destination' in df.columns:
    df.drop(columns=['Arrival Destination'], inplace=True)
if 'Station Name' in df.columns:
    df.drop(columns=['Station Name'], inplace=True)

# Link Transactions to Tickets

# Merge ticket dimension
df = pd.merge(
    df,
    df_tickets[['Ticket ID', 'Railcard', 'Ticket Class', 'Ticket Type']],
    on=['Railcard', 'Ticket Class', 'Ticket Type'],
    how='left'
)
df.rename(columns={'Ticket ID': 'Ticket ID FK'}, inplace=True)

# Safely drop text fields after foreign key added
for col in ['Railcard', 'Ticket Class', 'Ticket Type', 'Ticket']:
    if col in df.columns:
        df.drop(columns=col, inplace=True)

In [ ]:
# JOURNEYS (Fact Table - Operational)
df_journeys = df[[
    'Journey ID', 'Routes', 'Date of Journey', 'Day of Week', 'Departure Time', 'Arrival Time',
    'Actual Arrival Time', 'Journey Duration (Min)', 'Journey Status',
    'Delay Time', 'Delay in Minutes (Numeric)', 'Delay ID FK',
    'Dep ID', 'Arr ID'
]].copy()

# Rename FKs for clarity
df_journeys.rename(columns={
    'Delay ID FK': 'Delay ID',
    'Dep ID': 'Dep ID',
    'Arr ID': 'Arr ID',
    'Journey Duration (Min)': 'Scheduled Duration (Min)'
}, inplace=True)

In [ ]:
# TRANSACTIONS (Fact Table - Financial)

# Create: Revenue/Ticket = Price - Refund Amount
df['Revenue/Ticket'] = df['Price'] - df['Refund Amount']

# Select required columns from main DataFrame
df_transactions = df[[
    'Transaction ID', 'Date of Purchase', 'Purchase Type', 'Payment Method',
    'Price', 'Refund Amount', 'Revenue/Ticket', 'Journey ID', 'Refund Request',
    'Ticket ID FK'
]].copy()

# Create Dimension Table for Purchase Channel
df_purchase_channel = df[['Purchase Type', 'Payment Method']].drop_duplicates().reset_index(drop=True)
df_purchase_channel.insert(0, 'Channel ID', df_purchase_channel.index + 1)

# Merge to add Channel ID to transactions
df_transactions = pd.merge(
    df_transactions,
    df_purchase_channel,
    on=['Purchase Type', 'Payment Method'],
    how='left'
)

# Rename foreign keys for clarity
df_transactions.rename(columns={
    'Ticket ID FK': 'Ticket ID',
    'Channel ID': 'Purchase Channel ID'
}, inplace=True)

# Drop text fields to normalize table
for col in ['Purchase Type', 'Payment Method']:
    if col in df_transactions.columns:
        df_transactions.drop(columns=col, inplace=True)


In [ ]:
# DIMENSIONS (Simple cleanup/rename)
df_tickets.rename(columns={'Ticket': 'Ticket Summary'}, inplace=True)
df_purchase_channel.rename(columns={'Channel ID': 'Purchase Channel ID'}, inplace=True)

In [ ]:
df_delay_reasons

,Delay ID,Reason for Delay (Text),Category
0,1,Not Delayed,No Delay
1,2,Signal Failure,Infrastructure
2,3,Technical Issue,Rolling Stock
3,4,Weather,External Factors
4,5,Staffing,Operational
5,6,Staff Shortage,Operational
6,7,Signal failure,Infrastructure
7,8,Traffic,External Factors


In [ ]:
output_files = {
    'Journeys': df_journeys,
    'Transactions': df_transactions,
    'Delay Reasons': df_delay_reasons,
    'Stations': df_stations,
    'Tickets': df_tickets,
    'Purchase Channel': df_purchase_channel
}

for name, dtable in output_files.items():
    dtable.to_csv(f"{name}.csv", index=False)